In [1]:
import math
from operator import mul


%env JAX_PLATFORM_NAME=cuda

import warnings
from functools import partial, reduce

import jax
import jax.numpy as jnp
import jax.random as jr
import jax.tree_util as jtu
import numpy as np
import numpyro
import ot
import scipy
from jax import Array
from numpyro import diagnostics, distributions as dist
from numpyro.infer import MCMC, NUTS

from mcmc import run_lmc_numpyro


warnings.simplefilter("ignore", FutureWarning)

jnp.set_printoptions(precision=3, suppress=True)
jax.config.update("jax_enable_x64", True)
print(jax.devices("cuda"))


def get_model_and_data(data, name):
    dset = data[name][0, 0]
    x = dset["x"]
    labels = jnp.squeeze(dset["t"])
    # labels are -1 and 1, convert to 0 and 1
    labels = (labels + 1) / 2
    n, data_dim = x.shape
    print(f"Data shape: {x.shape}")

    # randomly shuffle the data
    perm = jax.random.permutation(jr.PRNGKey(0), n)
    x = x[perm]
    labels = labels[perm]

    n_train = min(int(n * 0.8), 1000)
    x_train = x[:n_train]
    labels_train = labels[:n_train]
    x_test = x[n_train:]
    labels_test = labels[n_train:]

    def model(x, labels):
        alpha = numpyro.sample("alpha", dist.Exponential(0.01))
        W = numpyro.sample("W", dist.Normal(jnp.zeros(data_dim), 1.0 / alpha))
        b = numpyro.sample("b", dist.Normal(jnp.zeros((1,)), 1.0 / alpha))
        logits = jnp.sum(W * x + b, axis=-1)
        return numpyro.sample("obs", dist.Bernoulli(logits=logits), obs=labels)

    return model, (x_train, labels_train, x_test, labels_test)


def compute_w2(x1, x2, num_iters):
    source_samples = np.array(x1)
    target_samples = np.array(x2)
    source_weights = np.ones(source_samples.shape[0]) / source_samples.shape[0]
    target_weights = np.ones(target_samples.shape[0]) / target_samples.shape[0]
    mm = ot.dist(source_samples, target_samples)
    return ot.emd2(source_weights, target_weights, mm, numItermax=num_iters)


@partial(jax.jit, static_argnames=("max_len",))
def energy_distance(x: Array, y: Array, max_len: int = 2**16):
    assert y.ndim == x.ndim
    assert x.shape[1:] == y.shape[1:]
    prod = reduce(mul, x.shape[1:], 1)
    if prod >= 4:
        max_len = int(max_len / math.sqrt(prod))

    if x.shape[0] > max_len:
        x = x[:max_len]
    if y.shape[0] > max_len:
        y = y[:max_len]

    @partial(jax.vmap, in_axes=(None, 0))
    def _dist_single(_x, _y_single):
        assert _x.ndim == _y_single.ndim + 1, f"{_x.ndim} != {_y_single.ndim + 1}"
        diff = _x - _y_single
        if x.ndim > 1:
            # take the norm over all axes except the first one
            diff = jnp.sqrt(jnp.sum(diff**2, axis=tuple(range(1, diff.ndim))))
        return jnp.mean(jnp.abs(diff))

    def dist(_x, _y):
        assert _x.ndim == _y.ndim
        return jnp.mean(_dist_single(_x, _y))

    return 2 * dist(x, y) - dist(x, x) - dist(y, y)


def dict_to_array(dct: dict):
    alpha = dct["alpha"]
    alpha = jnp.expand_dims(alpha, alpha.ndim)
    b = dct["b"]
    return jnp.concatenate([alpha, b, dct["W"]], axis=-1)


vec_dict_to_array = jax.jit(jax.vmap(dict_to_array, in_axes=0, out_axes=0))


def predict(x, samples):
    sum = jnp.sum(samples[:, 2:] * x + samples[:, 1:2], axis=-1)
    # apply sigmoid
    return 1.0 / (1.0 + jnp.exp(-sum))


def test_accuracy(x_test, labels_test, samples):
    if isinstance(samples, dict):
        samples = vec_dict_to_array(samples)
    sample_dim = samples.shape[-1]
    samples = jnp.reshape(samples, (-1, sample_dim))
    if samples.shape[0] > 2**10:
        samples = samples[: 2**10]

    func = jax.jit(jax.vmap(lambda x: predict(x, samples), in_axes=0, out_axes=0))
    predictions = func(x_test)
    assert predictions.shape == (
        labels_test.shape[0],
        samples.shape[0],
    ), f"{predictions.shape} != {(labels_test.shape[0], samples.shape[0])}"

    labels_test = jnp.reshape(labels_test, (labels_test.shape[0], 1))
    is_correct = jnp.abs(predictions - labels_test) < 0.5
    accuracy_per_sample = jnp.mean(is_correct, axis=0)

    avg_accuracy = jnp.mean(accuracy_per_sample)

    len10 = int(0.1 * accuracy_per_sample.shape[0])
    best_sorted = jnp.sort(accuracy_per_sample)[len10:]
    accuracy_best90 = jnp.mean(best_sorted)
    return avg_accuracy, accuracy_best90


def eval_logreg(
    samples,
    evals_per_sample=None,
    ground_truth=None,
    num_iters_w2=0,
    x_test=None,
    labels_test=None,
):
    if isinstance(samples, dict):
        samples = vec_dict_to_array(samples)

    sample_dim = samples.shape[-1]
    reshaped_with_alpha = jnp.reshape(samples, (-1, sample_dim))
    vars = jnp.var(reshaped_with_alpha, axis=0)
    means = jnp.mean(reshaped_with_alpha, axis=0)
    result_str = f"means: {means},\nvars:  {vars}"

    samples_with_alpha = samples
    samples = samples[..., 1:]
    reshaped = jnp.reshape(samples, (-1, sample_dim - 1))

    ess = diagnostics.effective_sample_size(samples)
    avg_ess = 1 / jnp.mean(1 / jnp.stack(jtu.tree_leaves(ess)))
    ess_per_sample = avg_ess / reshaped.shape[0]
    result_str += (
        f"\nEffective sample size: {avg_ess:.4},"
        f" ess per sample: {ess_per_sample:.4}"
    )
    if evals_per_sample is not None:
        result_str += f", grad evals per sample: {evals_per_sample:.4}"

    half_len = reshaped.shape[0] // 2
    energy_self = energy_distance(reshaped[:half_len], reshaped[half_len:])
    result_str += f"\nEnergy dist v self: {energy_self:.4}"

    if ground_truth is not None:
        ground_truth = ground_truth[..., 1:]
        energy_gt = energy_distance(reshaped, ground_truth)
        result_str += f", energy dist vs ground truth: {energy_gt:.4}"
    if num_iters_w2 > 0 and ground_truth is not None:
        w2 = compute_w2(reshaped, ground_truth, num_iters_w2)
        result_str += f", Wasserstein-2: {w2:.4}"

    if x_test is not None and labels_test is not None:
        acc_error, acc_best90 = test_accuracy(x_test, labels_test, samples_with_alpha)
        result_str += (
            f"\nTest_accuracy: {acc_error:.4}, top 90% accuracy: {acc_best90:.4}"
        )
    else:
        acc_error, acc_best90 = None, None

    print(result_str)

    result_dict = {
        "ess": avg_ess,
        "ess_per_sample": ess_per_sample,
        "energy_v_self": energy_self,
        "grad_evals_per_sample": evals_per_sample,
        "test_accuracy": acc_error,
        "top90_accuracy": acc_best90,
    }

    return result_str, result_dict


dataset = scipy.io.loadmat("mcmc_data/benchmarks.mat")
data_name = "banana"
model_logreg, data_split = get_model_and_data(dataset, data_name)
x_train, labels_train, x_test, labels_test = data_split

env: JAX_PLATFORM_NAME=cuda
[cuda(id=0)]
Data shape: (5300, 2)


In [2]:
file_name = f"mcmc_data/{data_name}_ground_truth.npy"

# gt_nuts = MCMC(NUTS(model_logreg, step_size=1.0), num_warmup=2**13, num_samples=2**16)
# gt_nuts.run(jr.PRNGKey(0), x_train, labels_train)
# gt_logreg = vec_dict_to_array(gt_nuts.get_samples())
# # thin the samples to 2**15
# np.save(file_name, gt_logreg)

gt_logreg = np.load(file_name)
gt_logreg = gt_logreg[::4]
size_gt_half = int(gt_logreg.shape[0] // 2)
energy_bias = energy_distance(gt_logreg[:size_gt_half], gt_logreg[size_gt_half:])
print(f"Energy bias: {energy_bias}")
print(f"Ground truth shape: {gt_logreg.shape}")
print(f"test accuracy: {test_accuracy(x_test, labels_test, gt_logreg)}")
flattened_gt = jnp.reshape(gt_logreg, (-1, 4))
print(flattened_gt.shape)
print(jnp.var(flattened_gt, axis=0))
print(jnp.mean(flattened_gt, axis=0))

Energy bias: 0.251009472403517
Ground truth shape: (16384, 4)
test accuracy: (Array(0.532, dtype=float32), Array(0.543, dtype=float32))


In [14]:
num_chains = 2**5
num_samples_per_chain = 2**10
warmup_len = 2**12

In [15]:
out_logreg_lmc, steps_logreg_lmc = run_lmc_numpyro(
    jr.PRNGKey(2),
    model_logreg,
    (x_train, labels_train),
    num_chains,
    num_samples_per_chain,
    chain_sep=0.1,
    tol=0.01,
    warmup_mult=warmup_len,
    warmup_tol_mult=16,
    use_adaptive=False,
)
out_logreg_lmc["alpha"] = jnp.exp(out_logreg_lmc["alpha"])
print(jtu.tree_map(lambda x: x.shape, out_logreg_lmc))

100.00%|██████████| [00:27<00:00,  3.66%/s]
100.00%|██████████| [01:06<00:00,  1.51%/s]

LMC: gradient evaluations per output: 32.08
{'W': (32, 1024, 2), 'alpha': (32, 1024), 'b': (32, 1024, 1)}


In [16]:
_ = eval_logreg(
    out_logreg_lmc, steps_logreg_lmc, x_test=x_test, labels_test=labels_test
)

means: [23.139 -0.088 -0.025 -0.022],
vars:  [2091.395    0.008    0.041    0.028]
Effective sample size: 7021.649662516795, grad evals per sample: 32.08
Energy dist v self: 0.0005181
Test_accuracy: 0.539, top 90% accuracy: 0.5494


In [8]:
nuts = MCMC(
    NUTS(model_logreg),
    num_warmup=warmup_len,
    num_samples=num_samples_per_chain,
    num_chains=num_chains,
    chain_method="vectorized",
)
nuts.warmup(
    jr.PRNGKey(2),
    x_train,
    labels_train,
    extra_fields=("num_steps",),
    collect_warmup=True,
)
warmup_steps = sum(nuts.get_extra_fields()["num_steps"])
nuts.run(jr.PRNGKey(2), x_train, labels_train, extra_fields=("num_steps",))
out_logreg_nuts = nuts.get_samples(group_by_chain=True)
num_steps_nuts = sum(nuts.get_extra_fields()["num_steps"]) + warmup_steps
geps_nuts = num_steps_nuts / (num_chains * num_samples_per_chain)
print(geps_nuts)
print(jtu.tree_map(lambda x: x.shape, out_logreg_nuts))

sample: 100%|██████████| 1024/1024 [00:06<00:00, 159.68it/s]


21.57537841796875


In [12]:
_ = eval_logreg(
    out_logreg_nuts, geps_nuts, gt_logreg, x_test=x_test, labels_test=labels_test
)

means: [58.19  -0.045 -0.008 -0.008],
vars:  [3766.227    0.002    0.002    0.002]
Effective sample size: 5715.352966006341
Energy dist vs self: 4.118e-05, energy dist vs ground truth: 7.018e-06, test_accuracy: 0.5263, top 90% accuracy: 0.537


In [2]:
import pickle


def run_logreg_dataset(name, results_filename=None, results_dict_filename=None):
    model_logreg, data_split = get_model_and_data(dataset, name)
    x_train, labels_train, x_test, labels_test = data_split

    num_chains = 2**5
    num_samples_per_chain = 2**11
    warmup_len = 2**13

    out_logreg_lmc, geps_lmc = run_lmc_numpyro(
        jr.PRNGKey(3),
        model_logreg,
        (x_train, labels_train),
        num_chains,
        num_samples_per_chain,
        chain_sep=0.2,
        tol=0.02,
        warmup_mult=warmup_len,
        warmup_tol_mult=32,
        use_adaptive=False,
    )
    out_logreg_lmc["alpha"] = jnp.exp(out_logreg_lmc["alpha"])

    eval_lmc_str, eval_lmc_dict = eval_logreg(
        out_logreg_lmc,
        geps_lmc,
        ground_truth=None,
        x_test=x_test,
        labels_test=labels_test,
    )

    nuts = MCMC(
        NUTS(model_logreg),
        num_warmup=warmup_len,
        num_samples=num_samples_per_chain,
        num_chains=num_chains,
        chain_method="vectorized",
    )
    nuts.warmup(
        jr.PRNGKey(2),
        x_train,
        labels_train,
        extra_fields=("num_steps",),
        collect_warmup=True,
    )
    warmup_steps = sum(nuts.get_extra_fields()["num_steps"])
    nuts.run(jr.PRNGKey(2), x_train, labels_train, extra_fields=("num_steps",))
    out_logreg_nuts = nuts.get_samples(group_by_chain=True)
    num_steps_nuts = sum(nuts.get_extra_fields()["num_steps"]) + warmup_steps
    geps_nuts = num_steps_nuts / (num_chains * num_samples_per_chain)
    print("NUTS:")
    eval_nuts_str, eval_nuts_dict = eval_logreg(
        out_logreg_nuts,
        geps_nuts,
        ground_truth=None,
        x_test=x_test,
        labels_test=labels_test,
    )

    # Compute energy distance between the two methods
    def flatten_samples(samples):
        if isinstance(samples, dict):
            samples = vec_dict_to_array(samples)
        # remove alpha
        samples = samples[..., 1:]
        return jnp.reshape(samples, (-1, samples.shape[-1]))

    lmc_flat = flatten_samples(out_logreg_lmc)
    nuts_flat = flatten_samples(out_logreg_nuts)

    energy_dist = energy_distance(lmc_flat, nuts_flat)
    print(f"Energy distance between LMC and NUTS: {energy_dist:.5}")

    if results_filename is not None:
        with open(results_filename, "a") as f:
            f.write(f"{name}\n")
            f.write(f"LMC: {eval_lmc_str}\n\n")
            f.write(f"NUTS: {eval_nuts_str}\n\n")
            f.write(f"Energy distance: {energy_dist:.5}\n\n\n")

    results_dict = {
        "dataset_name": name,
        "LMC": eval_lmc_dict,
        "NUTS": eval_nuts_dict,
        "Energy distance": energy_dist,
    }

    if results_dict_filename is not None:
        with open(results_dict_filename, "wb") as f:
            pickle.dump(results_dict, f)

In [3]:
names = [
    "banana",
    "breast_cancer",
    "diabetis",
    "flare_solar",
    "german",
    "heart",
    "image",
    "ringnorm",
    "splice",
    "thyroid",
    "titanic",
    "twonorm",
    "waveform",
]

In [4]:
# make a file for the results, which has date and time in the name
import datetime


time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
results_filename = f"mcmc_data/results_{time}.txt"
results_dict_filename = f"mcmc_data/results_dict_{time}.pkl"

# create the results file
with open(results_filename, "w") as f:
    f.write("Results\n\n")

# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
for name in names:
    print(f"==================== {name} ====================")
    run_logreg_dataset(name, results_filename, results_dict_filename)
    print()

==================== banana ====================
Data shape: (5300, 2)


100.00%|██████████| [00:15<00:00,  6.36%/s]
100.00%|██████████| [01:52<00:00,  1.12s/%]


means: [16.386 -0.105 -0.033 -0.052],
vars:  [344.705   0.031   0.02    0.025]
Effective sample size: 3.698e+04, ess per sample: 0.5642, grad evals per sample: 22.03
Energy dist v self: 4.406e-05
Test_accuracy: 0.5463, top 90% accuracy: 0.5535


sample: 100%|██████████| 2048/2048 [00:09<00:00, 221.97it/s]


NUTS:
means: [16.761 -0.101 -0.03  -0.047],
vars:  [162.443   0.001   0.002   0.003]
Effective sample size: 2.806e+04, ess per sample: 0.4282, grad evals per sample: 32.16
Energy dist v self: 9.073e-06
Test_accuracy: 0.5509, top 90% accuracy: 0.5549
Energy distance between LMC and NUTS: 0.00061195

==================== breast_cancer ====================
Data shape: (263, 9)


...

NUTS:
means: [0.862 0.005 1.459 1.337 1.468 1.074 0.936 0.768 0.997 1.259 1.278 1.241
 1.128 0.738 1.378 1.175 1.617 1.236 0.993 1.2   1.082 1.085],
vars:  [0.03  0.    0.119 0.102 0.128 0.121 0.079 0.084 0.092 0.104 0.085 0.093
 0.104 0.077 0.102 0.1   0.154 0.09  0.089 0.098 0.108 0.099]
Effective sample size: 3.365e+04, ess per sample: 0.5135, grad evals per sample: 41.59
Energy dist v self: 0.0003256
Test_accuracy: 0.9716, top 90% accuracy: 0.972
Energy distance between LMC and NUTS: 58.496

==================== waveform ====================
Data shape: (5000, 21)


100.00%|██████████| [00:16<00:00,  6.14%/s]
100.00%|██████████| [01:56<00:00,  1.17s/%]


means: [ 2.136 -0.85   0.344 -0.685  0.277 -0.043  1.806  0.88   2.31   1.766
  3.717  2.355  3.928  1.853  0.894 -1.279 -5.723 -3.477 -3.645 -2.942
 -1.027 -1.062 -0.391],
vars:  [  0.388  14.634   2.634   9.381   2.968   0.325  69.653  12.033 106.707
  65.995 282.082 113.241 301.536  68.579  17.672  33.537 622.113 237.352
 255.168 164.02   22.418  22.402   2.791]
Effective sample size: 765.6, ess per sample: 0.01168, grad evals per sample: 22.03
Energy dist v self: 0.01838
Test_accuracy: 0.8728, top 90% accuracy: 0.8751


sample: 100%|██████████| 2048/2048 [00:20<00:00, 102.39it/s]


NUTS:
means: [ 2.257 -0.152  0.034 -0.106  0.023  0.002  0.319  0.227  0.391  0.398
  0.714  0.553  0.695  0.311  0.104 -0.292 -0.852 -0.619 -0.673 -0.601
 -0.232 -0.199 -0.07 ],
vars:  [0.147 0.    0.011 0.012 0.014 0.019 0.026 0.029 0.032 0.03  0.028 0.022
 0.022 0.022 0.024 0.029 0.042 0.032 0.028 0.022 0.015 0.012 0.011]
Effective sample size: 6.79e+04, ess per sample: 1.036, grad evals per sample: 86.95
Energy dist v self: 0.0001559
Test_accuracy: 0.8749, top 90% accuracy: 0.8758
Energy distance between LMC and NUTS: 0.92776

